#CNN using Google Colab and Keras Tuner


In [ ]:
!pip install keras.tuner

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras.tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=4e0d584129e1abfa0d31aa6595cd38ea43eacf434c2546cad47a2f8406948a62
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=4fca87598c9a6157f9414a72edbd83458b6f4aac8792ef23e031d6726e238e8b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras.tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.2.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3757 - accuracy: 0.8644 - val_loss: 0.3236 - val_accuracy: 0.8808
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2363 - accuracy: 0.9132 - val_loss: 0.2526 - val_accuracy: 0.9080
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1746 - accuracy: 0.9351 - val_loss: 0.2452 - val_accuracy: 0.9142


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3789 - accuracy: 0.8609 - val_loss: 0.2901 - val_accuracy: 0.8918
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2347 - accuracy: 0.9122 - val_loss: 0.2391 - val_accuracy: 0.9108
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1727 - accuracy: 0.9349 - val_loss: 0.2642 - val_accuracy: 0.9075


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5588 - accuracy: 0.8084 - val_loss: 0.4069 - val_accuracy: 0.8497
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4086 - accuracy: 0.8487 - val_loss: 0.3874 - val_accuracy: 0.8538
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3805 - accuracy: 0.8586 - val_loss: 0.3874 - val_accuracy: 0.8542


Epoch 1/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.4048 - accuracy: 0.8538 - val_loss: 0.3029 - val_accuracy: 0.8893
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2517 - accuracy: 0.9065 - val_loss: 0.2540 - val_accuracy: 0.9082
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1945 - accuracy: 0.9273 - val_loss: 0.2397 - val_accuracy: 0.9150


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3982 - accuracy: 0.8561 - val_loss: 0.2839 - val_accuracy: 0.8933
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2542 - accuracy: 0.9067 - val_loss: 0.2768 - val_accuracy: 0.8968
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1955 - accuracy: 0.9266 - val_loss: 0.2467 - val_accuracy: 0.9095


INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        102464    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1982528   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 2,086,282
Trainable params: 2,086,282
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1469 - accuracy: 0.9447 - val_loss: 0.2644 - val_accuracy: 0.9097
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1109 - accuracy: 0.9577 - val_loss: 0.2786 - val_accuracy: 0.9132
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0811 - accuracy: 0.9699 - val_loss: 0.3041 - val_accuracy: 0.9132
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0612 - accuracy: 0.9770 - val_loss: 0.3596 - val_accuracy: 0.9152
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0504 - accuracy: 0.9819 - val_loss: 0.4178 - val_accuracy: 0.9078
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0378 - accuracy: 0.9860 - val_loss: 0.4243 - val_accuracy: 0.9103
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0290 - accuracy: 0.9898 - val_loss: 0.4768 - val_accuracy